In [1]:
import BioSimSpace as BSS
import sire as sr

/home/btan/miniconda3/envs/openbiosim/lib/python3.11/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
mols = sr.load("1UBQ")

Unzipping './1UBQ.pdb.gz'...


In [ ]:
mols[0]

Molecule( 1UBQ:2  num_atoms=660 num_residues=134 )

In [7]:
!pdb4amber --reduce --dry --add-missing-atoms -o 1UBQ_dry.pdb 1UBQ.pdb


Summary of pdb4amber for: 1UBQ.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None


In [14]:
protein = BSS.IO.readMolecules("1UBQ_dry.pdb")[0]
protein = BSS.Parameters.ff14SB(protein).getMolecule()

protein_copy = BSS.IO.readMolecules("1UBQ_dry.pdb")[0]
protein_copy = BSS.Parameters.ff14SB(protein_copy).getMolecule()

In [15]:
mapping = BSS.Align.matchAtoms(molecule0=protein, molecule1=protein_copy, roi=[2])

/home/btan/miniconda3/envs/openbiosim/lib/python3.11/site-packages/BioSimSpace/Align/_align.py:1398: UserWarning: Residues 2 between molecule0 and molecule1 have identical atomtypes, which means you are likely attempting to match two identical residues.
  _warnings.warn(


In [18]:
aligned = BSS.Align.rmsdAlign(molecule0=protein, molecule1=protein_copy, roi=[2])
merged_protein = BSS.Align.merge(aligned, protein_copy, mapping, roi=[2])

In [ ]:
merged_system = merged_protein.toSystem()

padding = 15 * BSS.Units.Length.angstrom
box_min, box_max = merged_system.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
box_sizes = [x + padding for x in box_size]

box, angles = BSS.Box.rhombicDodecahedronHexagon(max(box_sizes))
solvated_system = BSS.Solvent.tip3p(molecule=merged_system, box=box, angles=angles, ion_conc=0.15)

BSS.Stream.save(solvated_system, "1UBQ")